# PR9-2 Frequent Itemsets

## Opdracht

PR9-2: Great Outdoors wil graag weten welke producten vaak samen gekocht worden door klanten, door het bouwen van Frequent Itemsets met A-Priori-algoritme. Tip: merge eerst de tabellen 'product' en 'order_details' om een juiste tabel met brongegevens te krijgen waarop je het algoritme kan toepassen. 
- Pas waar nodig Dummy Encoding toe.
- Train het initiële algoritme.
- Experimenteer met meerdere support & confidence thresholds.
- Gebruik [deze webpagina](https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6) als inspiratie.

# Implementatie

## Library Imports

In [123]:
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn import metrics
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
import sqlite3
import csv
import pyodbc

from apyori import apriori

from mlxtend.frequent_patterns import apriori as mlx_priori
from mlxtend.preprocessing import TransactionEncoder


## Util methods

In [124]:

def horizontal_rule(func) : 
    def wrapped_rule() :
        print( "=" * 30)
        func()
        print( "=" * 30)
    return wrapped_rule

## Data inlezen en samenvoegen

### Databases laden

In [125]:
sales_con = None
crm_con = None
staff_con = None
GO_SALES_PRODUCT_FORECASTData = None
GO_SALES_INVENTORY_LEVELSData = None



try : 
        sales_con = sqlite3.connect("../Assets/Week 2/go_sales.sqlite")
        print("Connected to go_sales sqlite.")


        crm_con = sqlite3.connect("../Assets/Week 2/go_crm.sqlite")
        print("Connected to go_crm sqlite.")


        staff_con = sqlite3.connect("../Assets/Week 2/go_staff.sqlite")
        print("Connected to go_staff sqlite.")

        GO_SALES_PRODUCT_FORECASTData = pd.read_csv("../Assets/Week 2/GO_SALES_PRODUCT_FORECASTData.csv")
        print("Connected to GO_SALES_PRODUCT_FORECASTData")

        GO_SALES_INVENTORY_LEVELSData = pd.read_csv("../Assets/Week 2/GO_SALES_INVENTORY_LEVELSData.csv", header=0, index_col=False)
        print("Connected to GO_SALES_INVENTORY_LEVELSData.csv")


except sqlite3.Error as error: 
        print("Failed to read data from sqlite table", error)

DataWarehouse_con = None

try : 
        pass
except : 
        pass




Connected to go_sales sqlite.
Connected to go_crm sqlite.
Connected to go_staff sqlite.
Connected to GO_SALES_PRODUCT_FORECASTData
Connected to GO_SALES_INVENTORY_LEVELSData.csv


### Tabellen laden

In [126]:
global sales_branch
sales_branch = pd.read_sql_query("SELECT * FROM {}".format("sales_branch"),sales_con)
sales_branch = sales_branch[sales_branch.columns.drop(list(sales_branch.filter(regex="TRIAL")))]

global retailer_site
retailer_site = pd.read_sql_query("SELECT * FROM {}".format("retailer_site"),sales_con)
retailer_site = retailer_site[retailer_site.columns.drop(list(retailer_site.filter(regex="TRIAL")))]

global country
country = pd.read_sql_query("SELECT * FROM {}".format("country"),sales_con)
country = country[country.columns.drop(list(country.filter(regex="TRIAL")))]

global product
product = pd.read_sql_query("SELECT * FROM {}".format("product"),sales_con)
product = product[product.columns.drop(list(product.filter(regex="TRIAL")))]

global product_type
product_type = pd.read_sql_query("SELECT * FROM {}".format("product_type"),sales_con)
product_type = product_type[product_type.columns.drop(list(product_type.filter(regex="TRIAL")))]

global product_line
product_line = pd.read_sql_query("SELECT * FROM {}".format("product_line"),sales_con)
product_line = product_line[product_line.columns.drop(list(product_line.filter(regex="TRIAL")))]

global order_header
order_header = pd.read_sql_query("SELECT * FROM {}".format("order_header"),sales_con)
order_header = order_header[order_header.columns.drop(list(order_header.filter(regex="TRIAL")))]

global order_details
order_details = pd.read_sql_query("SELECT * FROM {}".format("order_details"),sales_con)
order_details = order_details[order_details.columns.drop(list(order_details.filter(regex="TRIAL")))]


### Dataframe opstellen

In [127]:
dataset = pd.merge(order_details, product, left_on="PRODUCT_NUMBER", how="inner", right_on="PRODUCT_NUMBER")



dataset["ORDER_DETAIL_CODE"] = dataset["ORDER_DETAIL_CODE"].astype(int)
dataset["PRODUCT_NUMBER"] = dataset["PRODUCT_NUMBER"] .astype(int)
dataset["ORDER_NUMBER"] = dataset["ORDER_NUMBER"].astype(int)

display(dataset.dtypes)


display(dataset)

ORDER_DETAIL_CODE     int32
ORDER_NUMBER          int32
PRODUCT_NUMBER        int32
QUANTITY              int64
UNIT_COST            object
UNIT_PRICE           object
UNIT_SALE_PRICE      object
INTRODUCTION_DATE    object
PRODUCT_TYPE_CODE    object
PRODUCTION_COST      object
MARGIN               object
PRODUCT_IMAGE        object
LANGUAGE             object
PRODUCT_NAME         object
DESCRIPTION          object
dtype: object

,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,100000,8462,96,16,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
1,100001,9111,96,20,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
2,100002,8451,96,24,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
3,100003,8453,96,18,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
4,100004,8439,96,20,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99995,8459,95,146,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."
43059,99996,8457,95,172,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."
43060,99997,9267,95,192,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."
43061,99998,8441,95,192,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."


## Apriori Algoritme

### Data vormgeven

In [128]:
ItemsetList = []

for order in dataset.ORDER_NUMBER.unique() :
    # print(f" ordernumber : {order}")
    
    orderdataframe = dataset.loc[(dataset["ORDER_NUMBER"] == order),:]
    Itemlist = []

    for index, row in orderdataframe.iterrows(): 
        
        Itemlist.append(row["PRODUCT_NAME"])
        # print(Itemlist)
    
    if len(Itemlist) == 1 : 
        pass
    else :
        ItemsetList.append(Itemlist)


print( len(ItemsetList))
print(*ItemsetList, sep="\n")


5018
['Compact Relief Kit', 'Hailstorm Steel Irons', 'Hailstorm Titanium Woods Set', 'Blue Steel Max Putter', 'Course Pro Golf and Tee Set', 'Course Pro Gloves', 'Polar Sun', 'BugShield Lotion Lite', 'BugShield Lotion', 'Sun Shelter 30']
['Compact Relief Kit', 'Deluxe Family Relief Kit', 'TrailChef Canteen', 'Star Peg', 'Hibernator', 'Hibernator Camp Cot', 'Canyon Mule Climber Backpack', 'Firefly 4', 'EverGlow Single', 'BugShield Natural', 'Sun Blocker']
['Compact Relief Kit', 'Deluxe Family Relief Kit', 'Hailstorm Titanium Woods Set', 'Lady Hailstorm Steel Woods Set', 'Canyon Mule Journey Backpack', 'Mountain Man Analog', 'BugShield Lotion', 'Sun Shelter 30', 'Sun Shield']
['Compact Relief Kit', 'Aloe Relief', 'Lady Hailstorm Titanium Irons', 'Course Pro Golf Bag', 'Mountain Man Digital', 'BugShield Spray', 'Sun Shelter 15']
['Compact Relief Kit', 'Deluxe Family Relief Kit', 'Calamine Relief', 'TrailChef Kitchen Kit', 'TrailChef Double Flame', 'TrailChef Kettle', 'TrailChef Utensils',

### Methode 1 : *Apriori Algorithm in Data Mining: Part 02*

In [129]:
association_rules = apriori(ItemsetList, min_support=0.02, min_confidence=0.20, min_lift=3, min_length=4)
results = list(association_rules)

print( len(results))

display(results[54])

resultsframe = pd.DataFrame(results)
display(resultsframe)
# for rule in results : 
#     print(f"{results[rule]}\n{"="*30}")

81


RelationRecord(items=frozenset({'Granite Grip', 'Granite Carabiner'}), support=0.021123953766440814, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Granite Carabiner'}), items_add=frozenset({'Granite Grip'}), confidence=0.3212121212121212, lift=3.8286043331173976), OrderedStatistic(items_base=frozenset({'Granite Grip'}), items_add=frozenset({'Granite Carabiner'}), confidence=0.2517814726840855, lift=3.828604333117397)])

,items,support,ordered_statistics
0,"(Blue Steel Max Putter, Blue Steel Putter)",0.021523,"[((Blue Steel Max Putter), (Blue Steel Putter)..."
1,"(Course Pro Golf Bag, Blue Steel Max Putter)",0.020128,"[((Blue Steel Max Putter), (Course Pro Golf Ba..."
2,"(Hailstorm Titanium Irons, Blue Steel Max Putter)",0.027501,"[((Blue Steel Max Putter), (Hailstorm Titanium..."
3,"(Hailstorm Titanium Woods Set, Blue Steel Max ...",0.023515,"[((Blue Steel Max Putter), (Hailstorm Titanium..."
4,"(Lady Hailstorm Titanium Irons, Blue Steel Max...",0.020925,"[((Blue Steel Max Putter), (Lady Hailstorm Tit..."
...,...,...,...
76,"(Hailstorm Titanium Irons, Lady Hailstorm Stee...",0.020128,"[((Hailstorm Titanium Irons), (Lady Hailstorm ..."
77,"(Hailstorm Titanium Irons, Lady Hailstorm Tita...",0.024312,"[((Hailstorm Titanium Irons), (Lady Hailstorm ..."
78,"(Hailstorm Titanium Irons, Lady Hailstorm Tita...",0.025707,"[((Hailstorm Titanium Irons), (Lady Hailstorm ..."
79,"(Lady Hailstorm Titanium Woods Set, Hailstorm ...",0.022320,"[((Hailstorm Titanium Woods Set), (Lady Hailst..."


RelationRecord(
    items=frozenset(
        {'Blue Steel Max Putter', 'Blue Steel Putter'}), 
        support=0.021522518931845355, 
        ordered_statistics=[OrderedStatistic
                                (items_base=frozenset({'Blue Steel Max Putter'}),
                                 items_add=frozenset({'Blue Steel Putter'}), 
                                 confidence=0.304225352112676, 
                                 lift=3.9345433425294023), 
                            OrderedStatistic(
                                items_base=frozenset({'Blue Steel Putter'}), 
                                items_add=frozenset({'Blue Steel Max Putter'}), 
                                confidence=0.2783505154639175, 
                                lift=3.9345433425294023)
                                ]
                                )

### Methode 2 : *Data Science - Apriori Algorithm in Python- Market Basket Analysis*

#### Get list of items to prepare transaction dataframe

In [130]:
# Get list of items to prepare transaction dataframe
product_dummies = pd.get_dummies(product.loc[:,["PRODUCT_NAME"]])

productlist = product["PRODUCT_NAME"].tolist()
product_header = product["PRODUCT_NAME"].tolist()
product_header.insert(0,  'TRANSACTION_ID')

print(productlist)
print(product_header)


['TrailChef Water Bag', 'TrailChef Utensils', 'Insect Bite Relief', 'Hailstorm Steel Irons', 'Hailstorm Titanium Irons', 'Lady Hailstorm Steel Irons', 'Lady Hailstorm Titanium Irons', 'Hailstorm Titanium Woods Set', 'Hailstorm Steel Woods Set', 'Lady Hailstorm Titanium Woods Set', 'Lady Hailstorm Steel Woods Set', 'Course Pro Putter', 'Star Lite', 'Blue Steel Putter', 'Blue Steel Max Putter', 'Course Pro Golf and Tee Set', 'Course Pro Umbrella', 'Course Pro Golf Bag', 'Course Pro Gloves', 'Star Dome', 'Star Gazer 2', 'Star Gazer 3', 'Star Gazer 6', 'Star Peg', 'Hibernator Lite', 'Hibernator', 'Hibernator Extreme', 'TrailChef Canteen', 'Hibernator Self - Inflating Mat', 'Hibernator Pad', 'Hibernator Pillow', 'Hibernator Camp Cot', 'Canyon Mule Climber Backpack', 'Canyon Mule Weekender Backpack', 'Canyon Mule Journey Backpack', 'Canyon Mule Extreme Backpack', 'Canyon Mule Cooler', 'Canyon Mule Carryall', 'TrailChef Kitchen Kit', 'Firefly Lite', 'Firefly Mapreader', 'Firefly 2', 'Firefly 

#### Format into dataframe

In [131]:
#format for dataframe

#make list with first row being columns
transaction_dataset = [product_header]

#for each element in ItemsetList, make a new list(this will be a row in the dataframe)
#and apply one-hot encoding(for each item in list, append 0 or 1
#                            depending on if item is the same as column name)
for count, transaction in enumerate(ItemsetList) : 
    # print(count, transaction)
    Transaction = [count]
    # print(Transaction, transaction)
    for product in productlist: 
        if product in transaction:
            Transaction.append(1)
        else : 
            Transaction.append(0)
    # print(Transaction)
    # print(len(Transaction))
        
    transaction_dataset.append(Transaction)


# print(*transaction_dataset, sep="\n")

transaction_dataframe = pd.DataFrame(transaction_dataset)
transaction_dataframe.columns = transaction_dataframe.iloc[0]
transaction_dataframe = transaction_dataframe[1:]

transaction_dataframe = transaction_dataframe.astype(int)

print(ItemsetList[0])


display(transaction_dataframe.dtypes)
display(transaction_dataframe)



    

['Compact Relief Kit', 'Hailstorm Steel Irons', 'Hailstorm Titanium Woods Set', 'Blue Steel Max Putter', 'Course Pro Golf and Tee Set', 'Course Pro Gloves', 'Polar Sun', 'BugShield Lotion Lite', 'BugShield Lotion', 'Sun Shelter 30']


0
TRANSACTION_ID              int32
TrailChef Water Bag         int32
TrailChef Utensils          int32
Insect Bite Relief          int32
Hailstorm Steel Irons       int32
                            ...  
Sun Shield                  int32
Compact Relief Kit          int32
Deluxe Family Relief Kit    int32
Calamine Relief             int32
Aloe Relief                 int32
Length: 116, dtype: object

,TRANSACTION_ID,TrailChef Water Bag,TrailChef Utensils,Insect Bite Relief,Hailstorm Steel Irons,Hailstorm Titanium Irons,Lady Hailstorm Steel Irons,Lady Hailstorm Titanium Irons,Hailstorm Titanium Woods Set,Hailstorm Steel Woods Set,...,BugShield Extreme,Sun Blocker,Sun Shelter Stick,Sun Shelter 15,Sun Shelter 30,Sun Shield,Compact Relief Kit,Deluxe Family Relief Kit,Calamine Relief,Aloe Relief
1,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
3,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,1,1,0,0
4,3,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
5,4,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5014,5013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5015,5014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5016,5015,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5017,5016,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
frequency = pd.DataFrame(transaction_dataframe.aggregate("sum"))

display(frequency)

,0
0,
TRANSACTION_ID,12587653
TrailChef Water Bag,462
TrailChef Utensils,437
Insect Bite Relief,324
Hailstorm Steel Irons,417
...,...
Sun Shield,356
Compact Relief Kit,274
Deluxe Family Relief Kit,209


#### Convert to list of lists

In [133]:
records = []
for i in range (0, 5018) : 
    records.append(
        [
            transaction_dataframe.values[i,j] for j in range (1,116)
        ]
    )

print(*records, sep="\n")

[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

#### Build Apriori model

In [134]:
association_rules = apriori(
    records, 
    min_support=0.3, 
    min_confidence=0.2, 
    min_lift=1, 
    min_length=2)
results = list(association_rules)

print(len(results))
print(results[0])

3
RelationRecord(items=frozenset({0}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({0}), confidence=1.0, lift=1.0)])


### Methode 3 : *The Apriori Algorithm in Python: Discover Associations in Data*

In [135]:
te = TransactionEncoder()
te_ary = te.fit(ItemsetList).transform(ItemsetList)

In [148]:
df = pd.DataFrame(te_ary, columns=te.columns_)
 
# Apply the Apriori algorithm
frequent_itemsets = mlx_priori(df, min_support=0.02, use_colnames=True)
 
# display(df)

frequent_dataframe = pd.DataFrame(frequent_itemsets)
frequent_dataframe['length'] = frequent_dataframe['itemsets'].apply(lambda x: len(x))

display(frequent_dataframe)

,support,itemsets,length
0,0.061379,(Aloe Relief),1
1,0.088282,(Bear Edge),1
2,0.078517,(Bear Survival Edge),1
3,0.070745,(Blue Steel Max Putter),1
4,0.077322,(Blue Steel Putter),1
...,...,...,...
202,0.022320,"(Lady Hailstorm Titanium Woods Set, Hailstorm ...",2
203,0.021722,"(Lady Hailstorm Titanium Irons, Lady Hailstorm...",2
204,0.023316,"(Sun Blocker, Sun Shelter 30)",2
205,0.021722,"(TrailChef Canteen, TrailChef Water Bag)",2
